In [1]:
import pandas as pd
import numpy as np
import random
from IPython.display import clear_output
import time

import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv2D, GlobalMaxPooling1D, MaxPooling2D, Flatten, Dense, Input, Concatenate
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

import torch
import ast

from transformers import BertTokenizer, TFBertModel


import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')


In [2]:
df1 = pd.read_csv('all_2k_tweets_embedded_bert.csv')

In [ ]:
df1.head()

In [ ]:
df1['TweetText'][0]

In [3]:
def clean_text(sam):
    # Remove the 'tf.Tensor(\n' and trailing ')' part if present
    sam = sam.replace('tf.Tensor(\n', '').replace(')', '')
    sam = sam.replace(', shape=(1, 768, dtype=float32', '')
    sam = sam.replace('[[ ', '[[')
    sam = sam.replace('\n', '')
    sam = sam.replace('  ', ' ')
    sam = sam.replace(' ', ',')
    sam = sam.replace(',,', ',')
    return sam

In [ ]:
def string_to_tensor(sam):
    sam = clean_text(sam)
    # Convert string representation to list using ast.literal_eval
    sam = ast.literal_eval(sam)
    # Convert list to tensor
    #tensor = torch.tensor(sam)
    return sam[0]

In [ ]:
def to_tensor(sam):
    return tf.convert_to_tensor(sam)

In [ ]:
bad_index = [48041,57710]

In [ ]:
df1 = df1.drop(index = bad_index)

In [ ]:
i = 57710
imax = len(df1)
#imax = 100

while i < imax:
    if i in bad_index:
        i += 1
    else:
        df1['TweetText'][i] = string_to_tensor(df1['TweetText'][i])
        clear_output()
        print(i)
    
        i += 1

In [ ]:
df1['TweetText'] = df1['TweetText'].apply(to_tensor)

In [ ]:
df3 = df1.copy()

In [ ]:
# one hot encoding
one_hot = pd.get_dummies(df3['Region'])
df3 = df3.join(one_hot)
df3 = df3.drop('Region', axis = 1)
df3

In [ ]:
x_dat = df3[['TweetText']]
y_dat = df3[df1['Region'].unique()]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_dat, y_dat, test_size = 0.3, random_state = 50)

## asdf

In [4]:
def conv_val(st):
    return float(st)

In [5]:
def string_to_array(embedding_string):
    new = clean_text(embedding_string)
    new = new.replace('[[','').replace(']]','')
    new = new.split(',')
    
    new_val = []
    for val in new:
        new_val.append(conv_val(val))

    out_array = np.array([new_val]).astype(np.float32)
    
    return out_array


In [6]:
def array_reshape(array):
    flat = array.reshape(array.shape[0], array.shape[1],1)
    
    return flat

In [7]:
def array_to_tensor(array):
    tens = tf.convert_to_tensor(array, dtype = tf.float32)
    return tens

In [8]:
def reg_conv(val):
    cols = df1['Region'].unique()
    out = np.zeros(7)
    for i in range(7):
        if val == cols[i]:
            out[i] = 1

    return out

In [41]:
df5 = df1.sample(n = 50, random_state = 100)
df5.reset_index(inplace = True)

In [42]:
# Apply the conversion to the DataFrame
df5['TweetText'] = df5['TweetText'].apply(string_to_array)

In [43]:
df5['TweetText'] = df5['TweetText'].apply(array_reshape)

df5['TweetText'] = df5['TweetText'].apply(array_to_tensor)

In [44]:
df5['Timestamp'] = df5['Timestamp'].apply(array_to_tensor)

In [45]:
df5['Region'] = df5['Region'].apply(reg_conv)


df5['Region'] = df5['Region'].apply(array_to_tensor)

In [ ]:
one_hot = pd.get_dummies(df5['Region'])
df5 = df5.join(one_hot)
df5 = df5.drop('Region', axis = 1)

In [46]:
x_dat = df5['TweetText']
y_dat = df5['Region']
timestamps = df5['Timestamp']

In [47]:
type(x_dat[0])

numpy.ndarray

In [48]:
# Input layers
bert_input = Input(shape=(x_dat[0].shape[0], x_dat[0].shape[1]), name='bert_input')
timestamp_input = Input(shape=(1,), name='timestamp_input')

# Convolutional layer for BERT embeddings
conv_layer = Conv1D(30, kernel_size=3, strides=2, activation='relu')(bert_input)
pool_layer = GlobalMaxPooling1D()(conv_layer)

# Concatenate the timestamp with the convolutional output
concatenated = Concatenate()([pool_layer, timestamp_input])

# Fully connected layers
dense1 = Dense(20, activation='relu')(concatenated)
dense2 = Dense(15, activation='relu')(dense1)
output = Dense(7, activation='softmax')(dense2)

# Define the model
#rmodel = Model(inputs=[bert_input, timestamp_input], outputs=output)
rmodel = Model(inputs=bert_input, outputs=output)

# Compile the model
rmodel.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
rmodel.summary()


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ bert_input          │ (None, 1, 768)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 0, 30)     │     69,150 │ bert_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 30)        │          0 │ conv1d_7[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ timestamp_input     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 31)        │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ timestamp_input[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 20)        │        640 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 15)        │        315 │ dense_18[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 7)         │        112 │ dense_19[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 70,217 (274.29 KB)

 Trainable params: 70,217 (274.29 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
rmodel.fit([x_dat, timestamps], y_dat, epochs=10, batch_size=32)
#rmodel.fit(x_dat, y_dat, epochs=10, batch_size=32)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

rmodel = Sequential()
rmodel.add(Conv1D(30, kernel_size=3, strides=2, activation='relu', input_shape = (1, 768)))
rmodel.add(GlobalMaxPooling1D())
rmodel.add(Flatten())
rmodel.add(Dense(20, activation='relu'))
rmodel.add(Dense(15, activation='relu'))
rmodel.add(Dense(7, activation='softmax'))

rmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
rmodel.summary()


history = rmodel.fit(x_dat, y_dat, epochs = 5, batch_size = 32)